In [1]:
import os
from glob import glob

In [2]:
#from dask.distributed import Client, progress
#client = Client(threads_per_worker=4, n_workers=20)

In [3]:
import numpy as np
import pandas as pd
from dask import delayed
from dask import compute

from root_pandas import read_root 
from ROOT import TFile

Welcome to JupyROOT 6.18/04


/home/jcordero/anaconda3/envs/CMSenv/lib/python3.6/site-packages/root_numpy/__init__.py:34: RuntimeWarning: ROOT 6.18/04 is currently active but you installed root_numpy against ROOT 6.05/02. Please consider reinstalling root_numpy for this ROOT version.
  RuntimeWarning)
/home/jcordero/anaconda3/envs/CMSenv/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.15.0 is currently installed but you installed root_numpy against numpy 1.12.1. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [4]:
import matplotlib.pyplot as plt

In [5]:
from Common.CommonHelper  import CommonHelper
from Samples.DataStack    import DataStack
from Samples.DataSample   import DataSample
from Samples.Event        import Event
from Samples.DataFile     import DataFile
from Plotter.Histo        import Histo
from Plotter.HistoSample  import HistoSample
from Plotter.HistoVar     import HistoVar

In [6]:
def getSamplesPath(path):
    return glob(os.path.join(path,"*[!.][!s][!h]"))

def getSampleFromPath(path):
    return path.split("/")[-1]

def getFileFromPath(path):
    return path.split("/")[-1]

def getFiles(path,sample):
    return glob(os.path.join(path,sample,"output*[!v_0]*"))

In [7]:
def isPathEmpty(path,samp):
    file = glob(os.path.join(path, getSampleFromPath(samp),"output*"))
    if file:
        return False
    else:
        return True

In [8]:
def SampleLoop(Sample, HStack):
    for file in Sample:
        for i,iEvent in enumerate(file):
            event = Event(iEvent)

            for histo in HStack:
                histo.fill( event.value(histo.name),
                            event.value('eventWeight')*Sample.getSF())
    return HStack

In [9]:
path = "/home/jcordero/CMS/data/data_2018/rereco/SMP_ZG/Files/mumug/VDask/"
chuncksize = 5000
era = "2018"

dataFiles  = ['DoubleMuon']
dataFiles += ['DoubleMuon_'+era+run for era in ['2016','2017','2018'] for run in ['A','B','C','D','E','F','G','Histo']]

In [10]:
dataFiles

['DoubleMuon',
 'DoubleMuon_2016A',
 'DoubleMuon_2016B',
 'DoubleMuon_2016C',
 'DoubleMuon_2016D',
 'DoubleMuon_2016E',
 'DoubleMuon_2016F',
 'DoubleMuon_2016G',
 'DoubleMuon_2016Histo',
 'DoubleMuon_2017A',
 'DoubleMuon_2017B',
 'DoubleMuon_2017C',
 'DoubleMuon_2017D',
 'DoubleMuon_2017E',
 'DoubleMuon_2017F',
 'DoubleMuon_2017G',
 'DoubleMuon_2017Histo',
 'DoubleMuon_2018A',
 'DoubleMuon_2018B',
 'DoubleMuon_2018C',
 'DoubleMuon_2018D',
 'DoubleMuon_2018E',
 'DoubleMuon_2018F',
 'DoubleMuon_2018G',
 'DoubleMuon_2018Histo']

In [11]:
DStack = DataStack()

for samp in getSamplesPath(path):
    if not isPathEmpty(path,samp) :
        if getSampleFromPath(samp) not in dataFiles:
            DStack.append( DataSample(path, getSampleFromPath(samp), era, chuncksize) )
    

In [12]:
def Var2PlotDict(part,var,ph):
    VarDict = []
    dirstruc = {}
    for p in part:
        for v in var:
            for gm in ph:
                dirstruc['part'] = p
                dirstruc['var']  = v
                dirstruc['ph']   = gm
                VarDict.append(dirstruc)
                
    return VarDict
    
def Var2Plot():
    PartVar = []
    VarDict = []
    
    parts = ["photonOne","leptonOne","leptonTwo"]
    var = ["Pt","M","Eta","Phi"]
    ph = ["","_EE","_EB"]
    PartVar += [p+v+gm for p in parts for v in var for gm in ph]
    VarDict += Var2PlotDict(parts,var,ph)
    
    parts = ["dilepton","llg"]
    var = ["Pt","M","DR","DEta","DPhi"]
    PartVar += [p+v+gm for p in parts for v in var for gm in ph]
    VarDict += Var2PlotDict(parts,var,ph)
    
    return PartVar,VarDict

In [13]:
def HVarStackckIni():
    HVarStack = HistoVar()
    _,vardict = Var2Plot()
    
    for var in vardict:
        print(var)
        HVarStack.append(Histo(variable = var))
        
    return HVarStack

In [17]:
var = {'part': 'leptonTwo', 'var': 'Phi', 'ph': '_EB'}
histo = Histo(variable = var)
histo._getRanges(part = var['part'],var = var['var'],ph = var['ph'])
histo._getBins(part = var['part'],var = var['var'],ph = var['ph'])

30.0

In [15]:
%%time
HSampleStack = HistoSample()
for sample in DStack.getSamples():
    print(sample)
    HSampleStack.append(SampleLoop(DStack[sample], HVarStackckIni()),name = sample)
    #HSampleStack.append(delayed(SampleLoop)(DStack[sample], HVarStack))
#compute(*HSampleStack)

WZTo2L2Q
{'part': 'leptonTwo', 'var': 'Phi', 'ph': '_EB'}


TypeError: object of type 'numpy.float64' has no len()

In [16]:
fig,ax = HSampleStack.plot(variable="dileptonM",log=True)
#HSampleStack.plot(log=True)
#HSampleStack.plot(log=False)

TypeError: 'NoneType' object is not iterable

<Figure size 720x720 with 0 Axes>

In [ ]:
def PLOT(x=np.arange(10)):
    plt.plot(x,x)
    ax = plt.gca()
    return ax

In [ ]:
for stk in HSampleStack.stack:
    print(stk.variable)

In [ ]:
plt.rc('figure',figsize=[10,10])
PLOT()

In [ ]:
plotting.plot(np.arange(90))
plotting.show()